In [4]:
import os
import subprocess

def compress_with_mozjpeg(image_path, quality=75):
    folder, filename = os.path.split(image_path)
    name, ext = os.path.splitext(filename)

    new_filename = f"new{name}.jpg"
    new_path = os.path.join(folder, new_filename)

    # Full path to your cjpeg.exe
    cjpeg_path = r"E:\removeBackground\mozjpeg_4.1.1_x64\mozjpeg_4.1.1_x64\shared\tools\cjpeg.exe"

    try:
        subprocess.run([
            cjpeg_path, "-quality", str(quality), "-progressive",
            "-outfile", new_path, image_path
        ], check=True)
        print(f"✅ Compressed image saved as: {new_path}")
    except Exception as e:
        print(f"❌ Error: {e}")

if __name__ == "__main__":
    path = input("Enter path of image: ").strip('"')
    compress_with_mozjpeg(path, quality=75)


Enter path of image:  frame_0317.jpg


✅ Compressed image saved as: newframe_0317.jpg


In [19]:
import os
import cv2
import numpy as np
import subprocess
from rembg import remove
from PIL import Image

# ====== CONFIG ======
CJPEG_PATH = r"E:\removeBackground\mozjpeg_4.1.1_x64\mozjpeg_4.1.1_x64\shared\tools\cjpeg.exe"

# ====== FUNCTIONS ======
def dilate_mask(mask, dilation_pixels=20):
    """Expand mask outward to keep full human"""
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(mask, kernel, iterations=dilation_pixels // 2)
    return dilated

def expand_bbox(x_min, y_min, x_max, y_max, image_w, image_h, tightness=1.0):
    """Adjust bbox tightness (1.0 = tightest, <1.0 = looser crop)"""
    w = x_max - x_min
    h = y_max - y_min
    cx = (x_min + x_max) // 2
    cy = (y_min + y_max) // 2

    scale = 1 / tightness
    new_w = int(w * scale)
    new_h = int(h * scale)

    x_min_new = max(0, cx - new_w // 2)
    x_max_new = min(image_w, cx + new_w // 2)
    y_min_new = max(0, cy - new_h // 2)
    y_max_new = min(image_h, cy + new_h // 2)

    return x_min_new, y_min_new, x_max_new, y_max_new

def process_image(image_path, tightness=1.0, dilation_pixels=20, bg_color=(255, 255, 255), quality=75):
    # Load image
    orig = Image.open(image_path).convert("RGBA")  # ensure alpha exists
    image_w, image_h = orig.size

    # Step 1: Rembg
    result = remove(orig)
    result_np = np.array(result)

    # Step 2: Get mask & dilate
    mask = result_np[:, :, 3]  # alpha channel
    mask = (mask > 0).astype(np.uint8) * 255
    mask = dilate_mask(mask, dilation_pixels)

    # Step 3: Find bounding box
    coords = cv2.findNonZero(mask)
    x, y, w, h = cv2.boundingRect(coords)
    x_min, y_min, x_max, y_max = expand_bbox(x, y, x + w, y + h, image_w, image_h, tightness)

    # Step 4: Crop
    cropped_img = np.array(orig)[y_min:y_max, x_min:x_max]

    # Step 5: Replace background (handles alpha correctly)
    if cropped_img.shape[2] == 4:  # RGBA
        bg = np.full(cropped_img.shape[:2] + (3,), bg_color, dtype=np.uint8)
        alpha = (cropped_img[:, :, 3] / 255.0)[:, :, None]
        final_img = (alpha * cropped_img[:, :, :3] + (1 - alpha) * bg).astype(np.uint8)
    else:  # already RGB
        final_img = cropped_img

    # Save as temporary PNG
    temp_png = os.path.splitext(image_path)[0] + "_temp.png"
    cv2.imwrite(temp_png, final_img)

    # Step 6: Compress with MozJPEG
    folder, filename = os.path.split(image_path)
    name, ext = os.path.splitext(filename)
    new_filename = f"new{name}.jpg"
    new_path = os.path.join(folder, new_filename)

    try:
        subprocess.run([
            CJPEG_PATH, "-quality", str(quality), "-progressive",
            "-outfile", new_path, temp_png
        ], check=True)
        print(f"✅ Processed and saved: {new_path}")
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        if os.path.exists(temp_png):
            os.remove(temp_png)

# ====== RUN ======
if __name__ == "__main__":
    path = input("Enter path of image: ").strip('"')
    tightness = float(input("Enter tightness (1.0 = tightest, <1.0 = looser): "))
    process_image(path, tightness=tightness, dilation_pixels=20, quality=50)


Enter path of image:  frame_0209.jpg
Enter tightness (1.0 = tightest, <1.0 = looser):  .01


*************** EP Error ***************
EP Error D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:490 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
✅ Processed and saved: newframe_0209.jpg
